In [ ]:
import json
import pandas as pd
import numpy as np

read review_train.json and randomly select 1000000 reviews as training data

In [ ]:
with open('review_train.json','r') as f:
    lines = f.readlines()
raw = list(map(json.loads,lines))
test = pd.DataFrame(raw).sample(n=1000000,random_state=123)
# with open('review_test.json','r') as f:
    #lines = f.readlines()
#raw = list(map(json.loads,lines))
#test = pd.DataFrame(raw)

In [ ]:
import gc
del lines
del raw
gc.collect()

In [ ]:
test = test[['stars','text']]

In [ ]:
stars = test.stars

transform emoticons to English words

In [ ]:
import re
def convert_emo(text):
    happy = r"(:-\)|:\)|:-\]|:\]|:3|:c\)|:>|8\)|8-\)|:o\)|=\)|:\}|:\^\)|:-\)\))" 
    laugh = r"(:‑D|:D|8‑D|8D|x‑D|xD|X‑D|XD|=‑D|=D|=‑3|=3|B\^D)" 
    sad = r"(:‑\(|:\(|:‑c|:c|:‑<|:<|:‑\[|:\[|:\{|;\(|>:\[)" 
    angry = r"(:-\|\||:@|>:\()" 
    cry = r"(:'‑\(|:'\()" 
    horror = r"(D:<|D:|D8|D;|D=|DX|D‑':|v\.v)" 
    surprise = r"(>:O|:‑O|:O|:‑o|:o|8‑0|O_O|o‑o|O_o|o_O|o_o|O-O)" 
    kiss = r"(:\*|:-\*|:\^\*|\( '\}\{'\))" 
    wink = r"(;‑\)|;\)|\*-\)|\*\)|;‑\]|;\]|;D|;\^\)|:‑,)" 
    skeptical = r"(>:\\\\|>:/|:‑/|:‑\.|:/|:\\\\|=/|=\\\\|:L|=L|:S|>\.<)" 
    text = re.sub(happy,' happy ',text)
    text = re.sub(laugh,' laugh ',text)
    text = re.sub(sad,' sad ',text)
    text = re.sub(angry,' angry ',text)
    text = re.sub(cry,' cry ',text)
    text = re.sub(horror,' horror ',text)
    text = re.sub(surprise,' surprise ',text)
    text = re.sub(kiss,' kiss ',text)
    text = re.sub(wink,' wink ',text)
    text = re.sub(skeptical,' skeptical ',text)
    return text

In [ ]:
test.text = test.text.apply(convert_emo)

remove \n _

In [ ]:
def remove_gangn(text):
    text = re.sub('\n',' ',text)
    return text

In [ ]:
test.text = test.text.apply(remove_gangn)

In [ ]:
def rm_underline(text):
    text = re.sub('_',' ',text.lower())
    return text

In [ ]:
test.text = test.text.apply(rm_underline)

In [ ]:
test.index = list(range(len(test)))

Replace the review with only punctuation to 'speechless'

In [ ]:
for i in range(len(test)):
    if not bool(re.search('[A-Za-z]',test.text.loc[i])):
        test.text.loc[i] = 'speechless'

expand abbreviation

In [ ]:
def convert_abb(text):
    text = re.sub('n\'t',' not',text)
    text = re.sub('\'m',' am',text)
    text = re.sub('\'s', ' is',text)
    text = re.sub('\'ve',' have',text)
    text = re.sub('\'d',' would',text)
    return text

In [ ]:
test.text = test.text.apply(convert_abb)

handle negation

In [ ]:
import re
def add_negation(text):
    negation = ['not', 'never', 'no']
    words = text.split()[:]
    flag = 0
    for index, word in enumerate(words):
        if(flag != 0):
            words[index] = 'not_' + words[index]
        if(re.match('.*[;,.?!]$', word)):
            flag = 0
        if(word in negation):
            flag = 1
    text = " ".join(words)
    return(text)

In [ ]:
test.text = test.text.apply(add_negation)

remove all punctuation

In [ ]:
def remove_punc(text):
    text = re.sub('[/*-?!:;,\.\'\"\)\(]', ' ', text)
    return text

In [ ]:
test.text = test.text.apply(remove_punc)

lemmatize

In [ ]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
def lemmatizer_all(text):
    new_text = []
    lemmatizer = WordNetLemmatizer()
    for word, tag in pos_tag(word_tokenize(text)):
        if tag.startswith('NN'):
            new_text.append(lemmatizer.lemmatize(word, pos='n'))
        elif tag.startswith('VB'):
            new_text.append(lemmatizer.lemmatize(word, pos='v'))
        elif tag.startswith('JJ'):
            new_text.append(lemmatizer.lemmatize(word, pos='a'))
        elif tag.startswith('R'):
            new_text.append(lemmatizer.lemmatize(word, pos='r'))
        else:
            new_text.append(word)
    return ' '.join(new_text)

In [ ]:
test.text = test.text.apply(lemmatizer_all)

In [ ]:
len(test)

remove no importance words

In [ ]:
no_importance_pos = ['IN','MD','PRP','PRP$','TO','WDT','WP','WP$','WRB']
no_importance_word = ['i','he','she','they','the','a','an','this','that','those','be','not_the','there','not_to','not_a','not_me','not_my',
                      'not_in','not_of','not_or','not_is','not_was','not_for','not_not_be','do_not','then','and','not','always','go','have',
                      'do','get','so','one','just','all','not_and','so', 'say','here','ever','not_get','not_i','not_that','not_this',
                      'not_not','guy','some','call','guy','re','one','not_not_even','then','and','not','always','go','have','do','get',
                      'so','one','just','all','not_and', 'say','here','ever','not_get','not_i','not_that','not_this','not_not','guy','some',
                      'guy','re','one', 'not_have','not_be','not_','not_it','not_be','not_you','not_on','even','not_you',
                      'not_on','up','ll','now','too','need', 'really','now','two']
only_text = []
index = []
for i in range(len(test)):
    try:
        only_text.append(' '.join(test.text.loc[i].split(' ')))
    except:
        index.append(i)
        pass

In [ ]:
new_text = []
index2 = []
for i in range(len(only_text)):
    try:
        retain = []
        POS = pos_tag(only_text[i].split(' '))
        for word in POS:
            if word[1] not in no_importance_pos and word[0] not in no_importance_word and len(word[0])!= 1:
                retain.append(word[0])
        new_text.append(' '.join(retain))
    except:
        index2.append(i)
        pass

extract phrases

In [ ]:
from gensim.models.phrases import Phrases, Phraser

In [ ]:
only_word = [sentence.split(' ') for sentence in new_text]

In [ ]:
ph = Phraser(Phrases(only_word)

In [ ]:
final_text = []
for i in range(len(only_word)):
    final_text.append(' '.join(ph[only_word[i]]))

In [ ]:
len(final_text)

In [ ]:
data = [stars,final_text]
data = pd.DataFrame(data)
data = data.T
data.rename(columns = {0:'stars',1:'text'},inplace = True)

In [ ]:
data.to_csv('train_process.csv',index = False)